In [64]:
import pandas as pd
from utils import *

In [65]:
excel_input = 'output_previous/output_1.xlsx'

In [66]:
import json

with open('json_files/scraping_base.json', 'r', encoding='utf-8') as json_file:
    loaded_scraping_base = json.load(json_file)

# Convert serialized dictionaries back to DataFrames
def convert_dict_to_df(d):
    if isinstance(d, list) and all(isinstance(i, dict) for i in d):
        return pd.DataFrame(d)
    elif isinstance(d, list):
        return [convert_dict_to_df(i) for i in d]
    elif isinstance(d, dict):
        return {k: convert_dict_to_df(v) for k, v in d.items()}
    else:
        return d

scraping_base = convert_dict_to_df(loaded_scraping_base)

In [67]:
material_remarks = {material: {'remarks': content['remarks'][['text']]} for material, content in scraping_base.items()}

In [68]:
material_remarks.keys()

dict_keys(['Abrasives', 'Fluorspar', 'Mercury', 'Silicon', 'Aluminum', 'Gallium', 'Mica', 'Silver', 'Antimony', 'Garnet', 'Molybdenum', 'Soda Ash', 'Arsenic', 'Gemstones', 'Nickel', 'Stone', 'Asbestos', 'Germanium', 'Niobium', 'Strontium', 'Barite', 'Gold', 'Nitrogen', 'Sulfur', 'Bauxite', 'Graphite', 'Talc', 'Beryllium', 'Gypsum', 'Peat', 'Tantalum', 'Bismuth', 'Hafnium', 'Perlite', 'Tellurium', 'Boron', 'Helium', 'Phosphate Rock', 'Thallium', 'Bromine', 'Indium', 'Platinum', 'Thorium', 'Cadmium', 'Iodine', 'Potash', 'Tin', 'Cement', 'Iron and Steel', 'Pumice', 'Titanium', 'Cesium', 'Iron Ore', 'Quartz', 'Tungsten', 'Chromium', 'Iron Oxide Pigments', 'Rare Earths', 'Vanadium', 'Clays', 'Kyanite', 'Rhenium', 'Vermiculite', 'Cobalt', 'Lead', 'Rubidium', 'Wollastonite', 'Copper', 'Lime', 'Salt', 'Yttrium', 'Diamond', 'Lithium', 'Sand and Gravel', 'Zeolites', 'Diatomite', 'Magnesium', 'Scandium', 'Zinc', 'Feldspar', 'Manganese', 'Selenium'])

In [69]:
# filtered_data

In [70]:
# Convert the DataFrames in the dictionary
serialize_dict(material_remarks)

# Save the transformed dictionary as JSON
with open('materials_data.json', 'w') as json_file:
    json.dump(material_remarks, json_file, indent=4)

In [71]:
ref_chem_compo = pd.read_excel('sub_materials_database.xlsx', sheet_name='sub_materials_database')

metric_conversion_df = pd.read_excel('sub_materials_database.xlsx', sheet_name='metric_conversion_factor')

In [72]:
# for material in material_remarks.keys():
#     # Convert the remarks to a DataFrame
#     material_sample = pd.DataFrame(material_remarks[material]['remarks'], columns=['text'])

#     # List of sub-material names from the reference_db
#     materials_list = ref_chem_compo['sub_material_name'].tolist()

#     # List of metric units from the metric_conversion_df
#     metrics_list = metric_conversion_df['Metric Mentioned'].tolist()

#     # Apply the functions to the 'text' column of material_sample
#     material_sample['chemical_composition'] = material_sample['text'].apply(lambda text: find_all_chemical_compositions(text, materials_list, ref_chem_compo))
#     material_sample['metric_conv_factor'] = material_sample['text'].apply(lambda text: find_metric_conversion_factor(text, metrics_list, metric_conversion_df))

#     # Update the material_remarks dictionary
#     material_remarks[material]['remarks'] = material_sample.to_dict(orient='records')

# # Save the updated dictionary to JSON
# with open('materials_data.json', 'w') as json_file:
#     json.dump(material_remarks, json_file, indent=4)

In [73]:
# material_sample

## match the remarks with the possible existing metrics, then match this with a number to dreb fih bach klchi yweli normalized by the end!

In [74]:
import pandas as pd
import shutil

# Copy the original Excel file to create an updated version
shutil.copyfile(excel_input, 'updated_materials_data.xlsx')

# Open the Excel file for updating
with pd.ExcelWriter('updated_materials_data.xlsx', mode='a', if_sheet_exists='replace') as writer:
    for material in material_remarks.keys():
        # Convert the remarks to a DataFrame
        material_sample = pd.DataFrame(material_remarks[material]['remarks'], columns=['text'])

        # List of sub-material names from the reference_db
        materials_list = ref_chem_compo['sub_material_name'].tolist()

        # List of metric units from the metric_conversion_df
        metrics_list = metric_conversion_df['Metric Mentioned'].tolist()

        # Apply the functions to the 'text' column of material_sample
        material_sample['chemical_composition'] = material_sample['text'].apply(
            lambda text: find_all_chemical_compositions(text, materials_list, ref_chem_compo))
        material_sample['metric_conv_factor'] = material_sample['text'].apply(
            lambda text: find_metric_conversion_factor(text, metrics_list, metric_conversion_df))

        # Read the corresponding sheet from the original Excel file
        sheet_name = f"{material}_raw_info"
        original_df = pd.read_excel(excel_input, sheet_name=sheet_name)

        # Ensure the new DataFrame (material_sample) has the same index as the original_df
        material_sample = material_sample.reindex(original_df.index)

        # Insert the new columns next to the 'remark' column
        original_df.insert(1, 'chemical_composition', material_sample['chemical_composition'])
        original_df.insert(2, 'metric_conv_factor', material_sample['metric_conv_factor'])

        # Write the updated sheet back to the Excel file
        original_df.to_excel(writer, sheet_name=sheet_name, index=False)